In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:

%cd /content/gdrive/My Drive/Colab Notebooks

/content/gdrive/My Drive/Colab Notebooks


In [8]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import sklearn.preprocessing as pp
from sklearn.preprocessing import StandardScaler
import sklearn.datasets as sk
import csv
import math
from decimal import Decimal
import random
from scipy.stats import mannwhitneyu
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.datasets import fetch_openml


In [9]:
@tf.function
def cross_entropy(logits, y):
    ce_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, logits))
    return ce_op

@tf.function
def mse(pred, y):
    mse_op = tf.reduce_mean(tf.square(pred - y))
    return mse_op

@tf.function
def loss_function(labels,predictions,CEscalar,MSEscalar):

  labels = tf.dtypes.cast(labels,tf.float64)
 
  CE_object = cross_entropy(predictions,labels)

  MSE_object = mse(predictions,labels)
 
 
  loss_object = (CEscalar*CE_object)+(MSEscalar*MSE_object)

  return loss_object

@tf.function
def model_test(features, labels,model,CEscalar,MSEscalar):
    predictions = model(features,training=False)
    t_loss = loss_function(labels, predictions, CEscalar, MSEscalar)
    test_loss(t_loss)
    test_acc(labels, predictions)
    


@tf.function
def model_train(features, labels,model,CEscalar,MSEscalar,optimizer):
    # Define the GradientTape context
    with tf.GradientTape() as tape:
        # Get the probabilities
        predictions = model(features, training=True)
        # Calculate the loss
        loss = loss_function(labels, predictions, CEscalar, MSEscalar)   
    # Get the gradients
    gradients = tape.gradient(loss, model.trainable_weights)


    # Update the loss and accuracy
    train_loss(loss)
    train_acc(labels, predictions)
    return gradients 
       

      
def PrintFunc(template):
    return template.format(epoch+1,loss, acc,tloss,tacc)

def shuffle(features,labels):
    #zip data
    data = list(zip(features,labels))

    #shuffle data
    #random.seed(1)
    #random.shuffle(data)
    np.random.shuffle(data)

    #unzip data
    features,labels = zip(*data)
    features = np.asarray(features)
    labels = np.asarray(labels)
    return features,labels  



def MeanAcrossEpoch(arr,num_epoch):
    
    tot = []
   
    arr = SameEpoch(arr,num_epoch)  
    for i in arr:
        val = 0
        val = np.mean(i,dtype=np.float64)
        tot.append(val)
    return tot      


def StdAcrossEpoch(arr,num_epoch):
    
    tot = []
   
    arr = SameEpoch(arr,num_epoch)  
    for i in arr:
        val = 0
        val = np.std(i,dtype=np.float64,ddof=1)
        tot.append(val)
    return tot    


def createList(r1, r2): 
  
    # Testing if range r1 and r2  
    # are equal 
    if (r1 == r2): 
        return r1 
  
    else: 
  
        # Create empty list 
        res = [] 
  
        # loop to append successors to  
        # list until r2 is reached. 
        while(r1 < r2+1 ): 
              
            res.append(r1) 
            r1 += 1
        return res 
          

def SameEpoch(arr,num_epoch):
    new = []
    for i in range(num_epoch):
        new.append(arr[i::num_epoch])   
    return new

#Get the mean for each epoch across the different folds
def MeanAcrossFolds(arr,num_epoch,num_folds):
    tot = []
    arr = SameEpoch(arr,num_epoch)  
    
    for i in range(num_epoch):
        val = 0
        val = np.sum(arr[i],dtype=np.float64)/num_folds
        tot.append(val)
    return tot       



#Whitney U test
def whitney(data,names,alpha=0.05,alt=None,savetoFile = False,file='default'):

  if savetoFile == False:

    for x in range(len(data)-1):
      print('\n')
      for y in range(x+1,len(data)):

        print(names[x] + ' Mean is less than ' + names[y])
        stat, p = mannwhitneyu(data[x], data[y] , alternative= alt)
        print('Statistics=%.3f, p=%.3f' % (stat, p))
        if p > alpha:
	        print('Same distribution (fail to reject H0)')
        else:
	        print('Different distribution (reject H0)')
  else:
      file = open("C:/Users/AutoMAttic/Desktop/honours/COS700/Research/glass"+file,'a')
      for x in range(len(data)-1):
        file.write('\n')
        file.write('\n')
        for y in range(x+1,len(data)):
          file.write(names[x] + ' Checking distibution ' + names[y])
          file.write('\n')
          stat, p = mannwhitneyu(data[x], data[y] , alternative= alt)
          file.write('Statistics=%.3f, p=%.3f' % (stat, p))
          file.write('\n')
          if p > alpha:
	          file.write('Same distribution (fail to reject H0)')
          else:
	          file.write('Different distribution (reject H0)')     
      file.close()


In [10]:
# Setting Environment
tf.keras.backend.set_floatx('float64')
tf.random.set_seed(1)
np.random.seed(1)
#random.seed(1)


# Hyper Parameter
LEARNING_RATE = 0.001
CE_SCALAR = 1
MSE_SCALAR = 0
NUM_EPOCHS = 100
BATCH_SIZE = 15
NUM_FOLDS = 10
RUNS = 30
TOTALEPOCHS = NUM_FOLDS*NUM_EPOCHS*RUNS

#Metrics
train_loss = tf.keras.metrics.Mean(name="train_loss")
test_loss = tf.keras.metrics.Mean(name='test_loss')
train_acc = tf.keras.metrics.CategoricalAccuracy(name="train_acc")
test_acc = tf.keras.metrics.CategoricalAccuracy(name="test_acc")


# Optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

# Load features and labels from iris
features, labels = sk.load_iris(return_X_y=True)

# Scale features
scaler = StandardScaler().fit(features)
features = scaler.transform(features)
labels = pp.label_binarize(labels, classes=np.unique(labels))

# shuffle data
features,labels=shuffle(features,labels)

#for stats test
StatsforHybrids = []

In [ ]:

#static hybrid functions
while(CE_SCALAR >= 0):

    seed = 1
    tf.random.set_seed(seed)
    np.random.seed(seed)
    

    #Declaring arrays for capturing all values during the program execution (For each fold,run and epoch)
    testAccSamples =[]
    trainAccSamples = []
    testLossSamples =[]
    trainLossSamples = []

    #Declaring arrays for holding mean values across folds
    MeantestAccSamples =[]
    MeantrainAccSamples = []
    MeantestLossSamples =[]
    MeantrainLossSamples = []

    #Declaring arrays for holding mean values across runs
    atr_accuracy_results = []
    ate_accuracy_results = []
    atr_loss_results = []
    ate_loss_results = []

    #Defining arrays to be used to flatten values into a one dimensional array 
    testAccflatten = []
    trainAccflatten = []
    testLossflatten = []
    trainLossflatten = []

    #arrays used to capture standard deviation 
    atrstdacc = []
    atestdacc = []
    atrstdloss = []
    atestdloss = []


    for run in tf.range(RUNS):

     
       

       #Constructing the model 
       model = tf.keras.Sequential([
                              tf.keras.layers.Flatten(input_shape=(4,)),
                              tf.keras.layers.Dense(4,use_bias=True,bias_initializer='ones', activation='relu'),
                              tf.keras.layers.Dense(3, activation='softmax'),
       ])



       #Save weights 
       Wsave = model.get_weights()
       print("run {:d}".format(run+1))

       
       fold=1 

       kfold = StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, random_state=74)
       # For data in number of folds
       for i, (train, test) in enumerate(kfold.split(features, labels.argmax(1))):

           X_train, X_test = features[train], features[test]
           y_train, y_test= labels[train], labels[test]    
           
           #set saved weights
           model.set_weights(Wsave) 
           
           print("Fold {:d}".format(fold))
           fold = fold + 1 
           
           #train for a number of epochs
           for epoch in tf.range(NUM_EPOCHS):
               
               #Shuffle dataset for each epoch
               tr = list(zip(X_train, y_train))
               te = list(zip(X_test, y_test))
               np.random.shuffle(tr)
               np.random.shuffle(te)
               X_train,y_train = zip(*tr) 
               X_test,y_test = zip(*te) 


               #Make numpy arrays to feed into model
               X_train=np.asarray(X_train, dtype=np.float64)
               y_train=np.asarray(y_train, dtype=np.float64)
               X_test=np.asarray(X_test, dtype=np.float64)
               y_test=np.asarray(y_test, dtype=np.float64)
               
               
               # Train on batch
               for i in tf.range(0,len(X_train) // BATCH_SIZE):
                   X = X_train[i * BATCH_SIZE:min(len(X_train), (i+1) * BATCH_SIZE)]
                   y = y_train[i * BATCH_SIZE:min(len(y_train), (i+1) * BATCH_SIZE)]
                   gradients=model_train(X, y,model,CE_SCALAR,MSE_SCALAR,optimizer)
                   optimizer.apply_gradients(zip(gradients, model.trainable_weights))   
                   
                      

               # Evaluate for each epoch    
               model_test(X_test, y_test,model,CE_SCALAR,MSE_SCALAR)   

               # Grab the results
               (loss, acc) = train_loss.result(), train_acc.result()
               (tloss, tacc) = test_loss.result(), test_acc.result()


               #Appending values for each epoch 
               testAccSamples.append(tacc)
               trainAccSamples.append(acc)
               testLossSamples.append(tloss)
               trainLossSamples.append(loss)


               # Clear the current state of the metrics
               train_loss.reset_states(), train_acc.reset_states()
               test_loss.reset_states(), test_acc.reset_states()

            
               # Local logging
               template = "Epoch {:03d}, loss: {:.3f}, acc: {:.3f}, test_loss {:.3f}, test_acc {:.3f}"
               output = PrintFunc(template)
               print(output)   
               
            
       


       #Using MeanAcrossFolds to get the mean for each epoch across the different folds
       testAccSamples = MeanAcrossFolds(testAccSamples ,NUM_EPOCHS,NUM_FOLDS)
       trainAccSamples = MeanAcrossFolds(trainAccSamples ,NUM_EPOCHS,NUM_FOLDS)
       testLossSamples  = MeanAcrossFolds(testLossSamples  ,NUM_EPOCHS,NUM_FOLDS)
       trainLossSamples = MeanAcrossFolds(trainLossSamples ,NUM_EPOCHS,NUM_FOLDS)
    
       #Appending fold mean values to arrays
       MeantestAccSamples.append(testAccSamples)
       MeantrainAccSamples.append(trainAccSamples)
       MeantestLossSamples.append(testLossSamples)
       MeantrainLossSamples.append(trainLossSamples)

       #Resetting Arrays to capture values for next run
       testAccSamples =[]
       trainAccSamples = []
       testLossSamples =[]
       trainLossSamples = []

       #Changing seed values for each run
       seed = run + 1
       tf.random.set_seed(seed)
       np.random.seed(seed)
       #random.seed(seed)
       



    
    #flattening the Mean values from all runs into a one dimensional array
    testAccflatten = [item for sublist in MeantestAccSamples for item in sublist]
    testAccflatten = SameEpoch(testAccflatten ,NUM_EPOCHS)

    trainAccflatten = [item for sublist in MeantrainAccSamples for item in sublist]
    trainAccflatten = SameEpoch(trainAccflatten ,NUM_EPOCHS)

    testLossflatten = [item for sublist in MeantestLossSamples for item in sublist]
    testLossflatten = SameEpoch(testLossflatten ,NUM_EPOCHS)

    trainLossflatten = [item for sublist in MeantrainLossSamples for item in sublist]
    trainLossflatten = SameEpoch(trainLossflatten ,NUM_EPOCHS)



    #Calculating the standard deviation 
    atrstdacc = StdAcrossEpoch(trainAccflatten,NUM_EPOCHS)
    atestdacc = StdAcrossEpoch(testAccflatten,NUM_EPOCHS)
    atrstdloss = StdAcrossEpoch(trainLossflatten,NUM_EPOCHS)
    atestdloss = StdAcrossEpoch(testLossflatten,NUM_EPOCHS)
    
    #Calculating the mean across epochs for the multiple runs
    atr_accuracy_results= MeanAcrossEpoch(trainAccflatten,NUM_EPOCHS)
    ate_accuracy_results= MeanAcrossEpoch(testAccflatten,NUM_EPOCHS)
    atr_loss_results= MeanAcrossEpoch(trainLossflatten,NUM_EPOCHS)
    ate_loss_results= MeanAcrossEpoch(testLossflatten,NUM_EPOCHS)



    file = open("IrisData.csv",'a')  
    file.write("CrossEntropy:"+str(CE_SCALAR) + " MeanSquared:"+str(MSE_SCALAR))
    file.write('\n')
    file.write('\n') 
    file.write("Acc Samples:")
    file.write('\n')
    for sample in testAccflatten[epoch]: 
        file.write(str(sample))
        file.write('\n')
    file.write('Test Acc mean:'+str(np.mean(testAccflatten[epoch],dtype = np.float64)))
    file.write('\n')
    file.write('Test Acc std:'+str(np.std(testAccflatten[epoch],ddof=1,dtype = np.float64)))
    file.write('\n')
    file.write('\n') 
    file.write("Loss Samples:")
    file.write('\n')
    for sample in testLossflatten[epoch]:        
        file.write(str(sample))
        file.write('\n')
    file.write('Test loss mean:'+str(np.mean(testLossflatten[epoch],dtype = np.float64)))
    file.write('\n')
    file.write('Test loss std:'+str(np.std(testLossflatten[epoch],ddof=1,dtype = np.float64)))
    file.write('\n')
    file.write('\n')
    file.write('\n')
    file.write('\n')
    file.write('\n')
    file.write('\n')
    file.write('\n')
    file.write('\n')
    file.write('\n')
    file.write('\n')
    file.close()  


    StatsforHybrids.append(testAccflatten[epoch])
 
    x = createList(0,NUM_EPOCHS-1)
    plt.clf()
    plt.ylabel("Acc", fontsize=14)
    plt.xlabel("Epoch", fontsize=14)
    plt.errorbar(x=x,y=ate_accuracy_results,yerr=atestdacc ,color='b', label = 'test',capsize=1, capthick=1, errorevery=2)
    plt.errorbar(x=x,y=atr_accuracy_results,yerr=atrstdacc ,color='r', label = 'train',capsize=1, capthick=1, errorevery=2)
        
    plt.legend(loc='lower right')
    plt.grid(True,color='k')
    #plt.show() 
    plt.savefig(f"CE{SCALAR1}MSE{SCALAR2}IrisAcc.eps", bbox_inches='tight')
    plt.clf()

    plt.ylabel("Loss", fontsize=14)
    plt.xlabel("Epoch", fontsize=14)
    plt.errorbar(x=x,y=ate_loss_results,yerr=atestdloss ,color='b', label = 'test', capsize=1, capthick=1, errorevery=2)
    plt.errorbar(x=x,y=atr_loss_results,yerr=atrstdloss,color='r', label = 'train', capsize=1, capthick=1, errorevery=2)

    plt.legend()
    plt.grid(True,color='k')
    #plt.show()
    plt.savefig(f"CE{CE_SCALAR}MSE{MSE_SCALAR}IrisLoss.eps", bbox_inches='tight')


    #Changing the hybrid for next run
    CE_SCALAR = CE_SCALAR - 0.25
    MSE_SCALAR = MSE_SCALAR + 0.25




In [12]:
  
#Adaptive hybrid CE to MSE
seed =1
tf.random.set_seed(1)
np.random.seed(1)

CE_SCALAR = 1
MSE_SCALAR = 0

#Declaring arrays for capturing all values during the program execution (For each fold,run and epoch)
testAccSamples =[]
trainAccSamples = []
testLossSamples =[]
trainLossSamples = []

#Declaring arrays for holding mean values across folds
MeantestAccSamples =[]
MeantrainAccSamples = []
MeantestLossSamples =[]
MeantrainLossSamples = []

#Declaring arrays for holding mean values across runs
atr_accuracy_results = []
ate_accuracy_results = []
atr_loss_results = []
ate_loss_results = []

#Defining arrays to be used to flatten values into a one dimensional array 
testAccflatten = []
trainAccflatten = []
testLossflatten = []
trainLossflatten = []

#arrays used to capture standard deviation 
atrstdacc = []
atestdacc = []
atrstdloss = []
atestdloss = []

seed =1

for run in range(RUNS):

     

    CEscalar = CE_SCALAR
    MSEscalar = MSE_SCALAR

  
    #Constructing the model 
    model = tf.keras.Sequential([
                              tf.keras.layers.Flatten(input_shape=(4,)),
                              tf.keras.layers.Dense(4,use_bias=True,bias_initializer='ones', activation='relu'),
                              tf.keras.layers.Dense(3, activation='softmax'),
       ])


    #Save weights 
    Wsave = model.get_weights()
    print("run {:d}".format(run+1))

    #Creating dataset for K-fold
    #dataset = ut.make_dataset(features,labels,NUM_FOLDS)
    fold=1 




    kfold = StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, random_state=74)

    #For data in number of folds 
    for i, (train, test) in enumerate(kfold.split(features, labels.argmax(1))):

        X_train, X_test = features[train], features[test]
        y_train, y_test= labels[train], labels[test]  

        CEscalar = CE_SCALAR
        MSEscalar = MSE_SCALAR  

        #set saved weights
        model.set_weights(Wsave)
       
        print("Fold {:d}".format(fold))
        fold = fold + 1 
           
        #train for a number of epochs
        for epoch in range(NUM_EPOCHS):


            tr = list(zip(X_train, y_train))
            te = list(zip(X_test, y_test))
            np.random.shuffle(tr)
            np.random.shuffle(te)
            X_train,y_train = zip(*tr) 
            X_test,y_test = zip(*te) 

            X_train=np.asarray(X_train, dtype=np.float64)
            y_train=np.asarray(y_train, dtype=np.float64)
            X_test=np.asarray(X_test, dtype=np.float64)
            y_test=np.asarray(y_test, dtype=np.float64)


            # Train on batch
            for i in range(0,len(X_train) // BATCH_SIZE):
                X = X_train[i * BATCH_SIZE:min(len(X_train), (i+1) * BATCH_SIZE)]
                y = y_train[i * BATCH_SIZE:min(len(y_train), (i+1) * BATCH_SIZE)]
                gradients=model_train(X, y,model,CEscalar,MSEscalar,optimizer)
                optimizer.apply_gradients(zip(gradients, model.trainable_weights))      

            # Evaluate for each epoch    
            model_test(X_test, y_test,model,CEscalar,MSEscalar)   
                 

            # Grab the results
            (loss, acc) = train_loss.result(), train_acc.result()
            (tloss, tacc) = test_loss.result(), test_acc.result()


            #Appending values for each epoch 
            testAccSamples.append(tacc)
            trainAccSamples.append(acc)
            testLossSamples.append(tloss)
            trainLossSamples.append(loss)


            #Clear the current state of the metrics
            train_loss.reset_states(), train_acc.reset_states()
            test_loss.reset_states(), test_acc.reset_states()

            
            #Local logging
            template = "Epoch {:03d}, loss: {:.3f}, acc: {:.3f}, test_loss {:.3f}, test_acc {:.3f}"
            output = PrintFunc(template)
            print(output)   
            CEscalar = CEscalar-0.01
            MSEscalar = MSEscalar + 0.01
            
            print('CE:%.3f  MSE:%.3f' % (CEscalar, MSEscalar))
            
       


    #Using MeanAcrossFolds to get the mean for each epoch across the different folds
    testAccSamples = MeanAcrossFolds(testAccSamples ,NUM_EPOCHS,NUM_FOLDS)
    trainAccSamples = MeanAcrossFolds(trainAccSamples ,NUM_EPOCHS,NUM_FOLDS)
    testLossSamples  = MeanAcrossFolds(testLossSamples  ,NUM_EPOCHS,NUM_FOLDS)
    trainLossSamples = MeanAcrossFolds(trainLossSamples ,NUM_EPOCHS,NUM_FOLDS)
    
    #Appending fold mean values to arrays
    MeantestAccSamples.append(testAccSamples)
    MeantrainAccSamples.append(trainAccSamples)
    MeantestLossSamples.append(testLossSamples)
    MeantrainLossSamples.append(trainLossSamples)

    #Resetting Arrays to capture values for next run
    testAccSamples =[]
    trainAccSamples = []
    testLossSamples =[]
    trainLossSamples = []

    #Changing seed values for each run
    seed = run + 1
    tf.random.set_seed(seed)
    np.random.seed(seed)
    



    
#flattening the Mean values from all runs into a one dimensional array
testAccflatten = [item for sublist in MeantestAccSamples for item in sublist]
testAccflatten = SameEpoch(testAccflatten ,NUM_EPOCHS)

trainAccflatten = [item for sublist in MeantrainAccSamples for item in sublist]
trainAccflatten = SameEpoch(trainAccflatten ,NUM_EPOCHS)

testLossflatten = [item for sublist in MeantestLossSamples for item in sublist]
testLossflatten = SameEpoch(testLossflatten ,NUM_EPOCHS)

trainLossflatten = [item for sublist in MeantrainLossSamples for item in sublist]
trainLossflatten = SameEpoch(trainLossflatten ,NUM_EPOCHS)



#Calculating the standard deviation 
atrstdacc = StdAcrossEpoch(trainAccflatten,NUM_EPOCHS)
atestdacc = StdAcrossEpoch(testAccflatten,NUM_EPOCHS)
atrstdloss = StdAcrossEpoch(trainLossflatten,NUM_EPOCHS)
atestdloss = StdAcrossEpoch(testLossflatten,NUM_EPOCHS)
    
#Calculating the mean across epochs for the multiple runs
atr_accuracy_results= MeanAcrossEpoch(trainAccflatten,NUM_EPOCHS)
ate_accuracy_results= MeanAcrossEpoch(testAccflatten,NUM_EPOCHS)
atr_loss_results= MeanAcrossEpoch(trainLossflatten,NUM_EPOCHS)
ate_loss_results= MeanAcrossEpoch(testLossflatten,NUM_EPOCHS)



file = open("/content/IrisDataAdCEtoMSE.csv",'a')  
file.write("CEtoMSE")
file.write('\n')
file.write('\n') 
file.write("Acc Samples:")
file.write('\n')
for sample in testAccflatten[epoch]: 
    file.write(str(sample))
    file.write('\n')
file.write('Test Acc mean:'+str(np.mean(testAccflatten[epoch],dtype = np.float64)))
file.write('\n')
file.write('Test Acc std:'+str(np.std(testAccflatten[epoch],ddof=1,dtype = np.float64)))
file.write('\n')
file.write('\n') 
file.write("Loss Samples:")
file.write('\n')
for sample in testLossflatten[epoch]:        
    file.write(str(sample))
    file.write('\n')
file.write('Test loss mean:'+str(np.mean(testLossflatten[epoch],dtype = np.float64)))
file.write('\n')
file.write('Test loss std:'+str(np.std(testLossflatten[epoch],ddof=1,dtype = np.float64)))
file.write('\n')
file.write('\n')
file.write('\n')
file.write('\n')
file.write('\n')
file.write('\n')
file.write('\n')
file.write('\n')
file.write('\n')
file.write('\n')
file.close()  


StatsforHybrids.append(testAccflatten[epoch])
 
x = createList(0,NUM_EPOCHS-1)
plt.clf()
plt.ylabel("Acc", fontsize=14)
plt.xlabel("Epoch", fontsize=14)
plt.errorbar(x=x,y=ate_accuracy_results,yerr=atestdacc ,color='b', label = 'test',capsize=1, capthick=1, errorevery=2)
plt.errorbar(x=x,y=atr_accuracy_results,yerr=atrstdacc ,color='r', label = 'train',capsize=1, capthick=1, errorevery=2)
        
plt.legend(loc='lower right')
plt.grid(True,color='k')
#plt.show() 
plt.savefig(f"/content/AccIrisAdaptiveCEtoMSE.eps", bbox_inches='tight')
plt.clf()

plt.ylabel("Loss", fontsize=14)
plt.xlabel("Epoch", fontsize=14)
plt.errorbar(x=x,y=ate_loss_results,yerr=atestdloss ,color='b', label = 'test', capsize=1, capthick=1, errorevery=2)
plt.errorbar(x=x,y=atr_loss_results,yerr=atrstdloss,color='r', label = 'train', capsize=1, capthick=1, errorevery=2)

plt.legend()
plt.grid(True,color='k')
#plt.show()
plt.savefig(f"/content/LossIrisAdaptiveCEtoMSE.eps", bbox_inches='tight')



run 1
Fold 1
Epoch 001, loss: 0.995, acc: 0.561, test_loss 1.152, test_acc 0.333
CE:0.990  MSE:0.010
Epoch 002, loss: 1.145, acc: 0.333, test_loss 1.095, test_acc 0.333
CE:0.980  MSE:0.020
Epoch 003, loss: 1.083, acc: 0.333, test_loss 1.032, test_acc 0.400
CE:0.970  MSE:0.030
Epoch 004, loss: 1.022, acc: 0.422, test_loss 0.974, test_acc 0.533
CE:0.960  MSE:0.040
Epoch 005, loss: 0.963, acc: 0.556, test_loss 0.929, test_acc 0.667
CE:0.950  MSE:0.050
Epoch 006, loss: 0.920, acc: 0.637, test_loss 0.893, test_acc 0.533
CE:0.940  MSE:0.060
Epoch 007, loss: 0.885, acc: 0.644, test_loss 0.862, test_acc 0.600
CE:0.930  MSE:0.070
Epoch 008, loss: 0.854, acc: 0.667, test_loss 0.834, test_acc 0.600
CE:0.920  MSE:0.080
Epoch 009, loss: 0.825, acc: 0.696, test_loss 0.807, test_acc 0.600
CE:0.910  MSE:0.090
Epoch 010, loss: 0.800, acc: 0.711, test_loss 0.783, test_acc 0.733
CE:0.900  MSE:0.100
Epoch 011, loss: 0.779, acc: 0.741, test_loss 0.762, test_acc 0.733
CE:0.890  MSE:0.110
Epoch 012, loss: 0.

KeyboardInterrupt: ignored

In [14]:
#Adaptive hybrid MSE to CE
seed =1
tf.random.set_seed(1)
np.random.seed(1)

CE_SCALAR = 0
MSE_SCALAR = 1

#Declaring arrays for capturing all values during the program execution (For each fold,run and epoch)
testAccSamples =[]
trainAccSamples = []
testLossSamples =[]
trainLossSamples = []

#Declaring arrays for holding mean values across folds
MeantestAccSamples =[]
MeantrainAccSamples = []
MeantestLossSamples =[]
MeantrainLossSamples = []

#Declaring arrays for holding mean values across runs
atr_accuracy_results = []
ate_accuracy_results = []
atr_loss_results = []
ate_loss_results = []

#Defining arrays to be used to flatten values into a one dimensional array 
testAccflatten = []
trainAccflatten = []
testLossflatten = []
trainLossflatten = []

#arrays used to capture standard deviation 
atrstdacc = []
atestdacc = []
atrstdloss = []
atestdloss = []

seed =1

for run in tf.range(RUNS):

     

    CEscalar = CE_SCALAR
    MSEscalar = MSE_SCALAR

  
   
    #Constructing the model 
    model = tf.keras.Sequential([
                              tf.keras.layers.Flatten(input_shape=(4,)),
                              tf.keras.layers.Dense(4,use_bias=True,bias_initializer='ones', activation='relu'),
                              tf.keras.layers.Dense(3, activation='softmax'),
       ])



    #Save weights 
    Wsave = model.get_weights()
    print("run {:d}".format(run+1))

    #Creating dataset for K-fold
    #dataset = ut.make_dataset(features,labels,NUM_FOLDS)
    fold=1 




    kfold = StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, random_state=74)

    #For data in number of folds 
    for i, (train, test) in enumerate(kfold.split(features, labels.argmax(1))):

        X_train, X_test = features[train], features[test]
        y_train, y_test= labels[train], labels[test]  

        CEscalar = CE_SCALAR
        MSEscalar = MSE_SCALAR  

        #set saved weights
        model.set_weights(Wsave)
       
        print("Fold {:d}".format(fold))
        fold = fold + 1 
           
        #train for a number of epochs
        for epoch in tf.range(NUM_EPOCHS):


            tr = list(zip(X_train, y_train))
            te = list(zip(X_test, y_test))
            np.random.shuffle(tr)
            np.random.shuffle(te)
            X_train,y_train = zip(*tr) 
            X_test,y_test = zip(*te) 

            X_train=np.asarray(X_train, dtype=np.float64)
            y_train=np.asarray(y_train, dtype=np.float64)
            X_test=np.asarray(X_test, dtype=np.float64)
            y_test=np.asarray(y_test, dtype=np.float64)


            # Train on batch
            for i in tf.range(0,len(X_train) // BATCH_SIZE):
                X = X_train[i * BATCH_SIZE:min(len(X_train), (i+1) * BATCH_SIZE)]
                y = y_train[i * BATCH_SIZE:min(len(y_train), (i+1) * BATCH_SIZE)]
                gradients=model_train(X, y,model,CEscalar,MSEscalar,optimizer)
                optimizer.apply_gradients(zip(gradients, model.trainable_weights))      

            # Evaluate for each epoch    
            model_test(X_test, y_test,model,CEscalar,MSEscalar)   
                 

            # Grab the results
            (loss, acc) = train_loss.result(), train_acc.result()
            (tloss, tacc) = test_loss.result(), test_acc.result()


            #Appending values for each epoch 
            testAccSamples.append(tacc)
            trainAccSamples.append(acc)
            testLossSamples.append(tloss)
            trainLossSamples.append(loss)


            #Clear the current state of the metrics
            train_loss.reset_states(), train_acc.reset_states()
            test_loss.reset_states(), test_acc.reset_states()

            
            #Local logging
            template = "Epoch {:03d}, loss: {:.3f}, acc: {:.3f}, test_loss {:.3f}, test_acc {:.3f}"
            output = PrintFunc(template)
            print(output)   
            CEscalar = CEscalar+0.01
            MSEscalar = MSEscalar - 0.01
            
            print('CE:%.3f  MSE:%.3f' % (CEscalar, MSEscalar))
            
       


    #Using MeanAcrossFolds to get the mean for each epoch across the different folds
    testAccSamples = MeanAcrossFolds(testAccSamples ,NUM_EPOCHS,NUM_FOLDS)
    trainAccSamples = MeanAcrossFolds(trainAccSamples ,NUM_EPOCHS,NUM_FOLDS)
    testLossSamples  = MeanAcrossFolds(testLossSamples  ,NUM_EPOCHS,NUM_FOLDS)
    trainLossSamples = MeanAcrossFolds(trainLossSamples ,NUM_EPOCHS,NUM_FOLDS)
    
    #Appending fold mean values to arrays
    MeantestAccSamples.append(testAccSamples)
    MeantrainAccSamples.append(trainAccSamples)
    MeantestLossSamples.append(testLossSamples)
    MeantrainLossSamples.append(trainLossSamples)

    #Resetting Arrays to capture values for next run
    testAccSamples =[]
    trainAccSamples = []
    testLossSamples =[]
    trainLossSamples = []

    #Changing seed values for each run
    seed = run + 1
    tf.random.set_seed(seed)
    np.random.seed(seed)
    



    
#flattening the Mean values from all runs into a one dimensional array
testAccflatten = [item for sublist in MeantestAccSamples for item in sublist]
testAccflatten = SameEpoch(testAccflatten ,NUM_EPOCHS)

trainAccflatten = [item for sublist in MeantrainAccSamples for item in sublist]
trainAccflatten = SameEpoch(trainAccflatten ,NUM_EPOCHS)

testLossflatten = [item for sublist in MeantestLossSamples for item in sublist]
testLossflatten = SameEpoch(testLossflatten ,NUM_EPOCHS)

trainLossflatten = [item for sublist in MeantrainLossSamples for item in sublist]
trainLossflatten = SameEpoch(trainLossflatten ,NUM_EPOCHS)



#Calculating the standard deviation 
atrstdacc = StdAcrossEpoch(trainAccflatten,NUM_EPOCHS)
atestdacc = StdAcrossEpoch(testAccflatten,NUM_EPOCHS)
atrstdloss = StdAcrossEpoch(trainLossflatten,NUM_EPOCHS)
atestdloss = StdAcrossEpoch(testLossflatten,NUM_EPOCHS)
    
#Calculating the mean across epochs for the multiple runs
atr_accuracy_results= MeanAcrossEpoch(trainAccflatten,NUM_EPOCHS)
ate_accuracy_results= MeanAcrossEpoch(testAccflatten,NUM_EPOCHS)
atr_loss_results= MeanAcrossEpoch(trainLossflatten,NUM_EPOCHS)
ate_loss_results= MeanAcrossEpoch(testLossflatten,NUM_EPOCHS)



file = open("/content/IrisDataAdMSEtoCE.csv",'a')  
file.write("MSEtoCE")
file.write('\n')
file.write('\n') 
file.write("Acc Samples:")
file.write('\n')
for sample in testAccflatten[epoch]: 
    file.write(str(sample))
    file.write('\n')
file.write('Test Acc mean:'+str(np.mean(testAccflatten[epoch],dtype = np.float64)))
file.write('\n')
file.write('Test Acc std:'+str(np.std(testAccflatten[epoch],ddof=1,dtype = np.float64)))
file.write('\n')
file.write('\n') 
file.write("Loss Samples:")
file.write('\n')
for sample in testLossflatten[epoch]:        
    file.write(str(sample))
    file.write('\n')
file.write('Test loss mean:'+str(np.mean(testLossflatten[epoch],dtype = np.float64)))
file.write('\n')
file.write('Test loss std:'+str(np.std(testLossflatten[epoch],ddof=1,dtype = np.float64)))
file.write('\n')
file.write('\n')
file.write('\n')
file.write('\n')
file.write('\n')
file.write('\n')
file.write('\n')
file.write('\n')
file.write('\n')
file.write('\n')
file.close()  


StatsforHybrids.append(testAccflatten[epoch])
 
x = createList(0,NUM_EPOCHS-1)
plt.clf()
plt.ylabel("Acc", fontsize=14)
plt.xlabel("Epoch", fontsize=14)
plt.errorbar(x=x,y=ate_accuracy_results,yerr=atestdacc ,color='b', label = 'test',capsize=1, capthick=1, errorevery=2)
plt.errorbar(x=x,y=atr_accuracy_results,yerr=atrstdacc ,color='r', label = 'train',capsize=1, capthick=1, errorevery=2)
        
plt.legend(loc='lower right')
plt.grid(True,color='k')
#plt.show() 
plt.savefig(f"/content/AccIrisAdaptiveMSEtoCE.eps", bbox_inches='tight')
plt.clf()

plt.ylabel("Loss", fontsize=14)
plt.xlabel("Epoch", fontsize=14)
plt.errorbar(x=x,y=ate_loss_results,yerr=atestdloss ,color='b', label = 'test', capsize=1, capthick=1, errorevery=2)
plt.errorbar(x=x,y=atr_loss_results,yerr=atrstdloss,color='r', label = 'train', capsize=1, capthick=1, errorevery=2)

plt.legend()
plt.grid(True,color='k')
#plt.show()
plt.savefig(f"/content/LossIrisAdaptiveMSEtoCE.eps", bbox_inches='tight')

run 1
Fold 1
Epoch 001, loss: 0.347, acc: 0.613, test_loss 0.192, test_acc 0.400
CE:0.010  MSE:0.990
Epoch 002, loss: 0.188, acc: 0.615, test_loss 0.164, test_acc 0.867
CE:0.020  MSE:0.980
Epoch 003, loss: 0.163, acc: 0.748, test_loss 0.156, test_acc 0.667
CE:0.030  MSE:0.970
Epoch 004, loss: 0.155, acc: 0.763, test_loss 0.155, test_acc 0.667
CE:0.040  MSE:0.960
Epoch 005, loss: 0.152, acc: 0.785, test_loss 0.152, test_acc 0.733
CE:0.050  MSE:0.950
Epoch 006, loss: 0.150, acc: 0.800, test_loss 0.151, test_acc 0.800
CE:0.060  MSE:0.940
Epoch 007, loss: 0.151, acc: 0.807, test_loss 0.151, test_acc 0.800
CE:0.070  MSE:0.930
Epoch 008, loss: 0.152, acc: 0.822, test_loss 0.153, test_acc 0.800
CE:0.080  MSE:0.920
Epoch 009, loss: 0.155, acc: 0.837, test_loss 0.156, test_acc 0.800
CE:0.090  MSE:0.910
Epoch 010, loss: 0.158, acc: 0.837, test_loss 0.160, test_acc 0.800
CE:0.100  MSE:0.900
Epoch 011, loss: 0.161, acc: 0.844, test_loss 0.164, test_acc 0.800
CE:0.110  MSE:0.890
Epoch 012, loss: 0.

KeyboardInterrupt: ignored

In [ ]:
print(StatsforHybrids)
Hybrids = ["CE1MSE0","CE75MSE25","CE5MSE5","CE25MSE75","CE0MSE1","CEtoMSE","MSEtoCE"]
whitney(StatsforHybrids,str1,0.05,alt=None,savetoFile=True,file="IrisStats.csv")